<a href="https://colab.research.google.com/github/likhitha-d01/object_detection/blob/master/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
PART-1

This is the notebook to preprocess the images and create the tfrecord files for training

Ensure the complete the following steps before proceeding

Steps:
1. Create a folder 'void_detector', and ensure that you change the cwd path in the 3rd cell. This will be the main folder
2. Download the following repository  --  #https://github.com/likhitha-d01/object_detection    -- , place in the main folder, rename it to object_detection
3. Create a 'data_folders' folder
4. Make annotations using imglab and place the files 'void.xml' , 'no_void.xml' in the 'data_folders' folder
5. Upload the images to './data_folders/full_data'

"""

"\nPART-1\n\nThis is the notebook to preprocess the images and create the tfrecord files for training\n\nEnsure the complete the following steps before proceeding\n\nSteps:\n1. Create a folder 'void_detector', and ensure that you change the cwd path in the 3rd cell. This will be the main folder\n2. Download the following repository  --  #https://github.com/likhitha-d01/object_detection    -- , place in the main folder, rename it to object_detection\n3. Create a 'data_folders' folder\n4. Make annotations using imglab and place the files 'void.xml' , 'no_void.xml' in the 'data_folders' folder\n5. Upload the images to './data_folders/full_data'\n\n"

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Pre-requisite libraries

!pip install protobuf
!pip install pillow
!pip install lxml
!pip install Cython
!pip install jupyter
!pip install matplotlib
!pip install pandas
!pip install opencv-python 
!pip install tensorflow==1.15.0

In [0]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
import requests 
import xml.etree.ElementTree as ET 
import shutil
from PIL import Image
import glob
import pandas as pd

cwd = '/content/drive/My Drive/void_detector'
print(os.path.exists(cwd))

!ls drive/My\ Drive/void_detector/data_folders

True
full_data    resized_imgs  test_im_s	train_im_s  val_im_s
no_void.xml  test_im_bw    train_im_bw	val_im_bw   void.xml


In [0]:
#Resizing the images to a common size

path = os.path.join(cwd,'data_folders')
dataset = os.path.join(path,'full_data')
des = os.path.join(path,'resized_imgs')
if os.path.exists(des)!=True:
	os.mkdir(des)

dim = (416,416)
files=[f for f in os.listdir(dataset) if '.jpg' in f]


if os.path.exists(des)!=True:
    print('Starting to resize the images..')
    for j,i in enumerate(files):
        print(j)
        image = cv2.imread(os.path.join(dataset,i))
        correct = np.array(255 * (image/ 255) ** 1.2 , dtype='uint8')
        cover = cv2.resize(correct,dim,interpolation = cv2.INTER_AREA)
        cv2.imwrite(os.path.join(des,i),cover)
    print('Finished resizing!')
if os.path.exists(des)==True:
  print('Images are already resized!')

Images are already resized!


In [0]:
#Shuffling the images and splitting to train , test and validation set and preprocessing the image

data_path = os.path.join(path,'resized_imgs')
filenames = [k for k in os.listdir(data_path) if '.jpg' in k]

train_files,split_files = train_test_split(filenames,test_size=0.1,shuffle=True)
val_files,test_files = train_test_split(split_files,test_size=0.5,shuffle=True)

destination =path
print(len(train_files))
print(len(val_files))
print(len(test_files))
files=[train_files,val_files,test_files]

data=['train_files','val_files','test_files']
des=['train_im_s','val_im_s','test_im_s']
c_dir = ['train_im_bw','val_im_bw','test_im_bw']


for j in range(len(des)):
    if os.path.exists(destination+'/'+des[j])!=True:
        os.mkdir(destination+'/'+des[j])
    if os.path.exists(destination+'/'+c_dir[j])!=True:
        os.mkdir(destination+'/'+c_dir[j])
    print('Starting to copy the {}'.format(data[j]))
    for names in files[j]:
        shutil.copy(data_path+'/'+names,destination+'/'+des[j])
    print('Finished copying the {}'.format(data[j]))

directory = path
dim = (416,416)

for k in range(len(c_dir)):
    num=0

    for file in os.listdir(directory+'/'+des[k]):
      print(directory+'/'+des[k]+'/'+file)
      num+=1
      print(num)
      image=cv2.imread(directory+'/'+des[k]+'/'+file)
      cv2.imwrite(directory+'/'+c_dir[k]+'/'+file,image)

1742
97
97
Starting to copy the train_files
Finished copying the train_files
Starting to copy the val_files
Finished copying the val_files
Starting to copy the test_files
Finished copying the test_files
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_8411.jpg
1
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_8215.jpg
2
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_9325.jpg
3
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_7540.jpg
4
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_8451.jpg
5
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_8814.jpg
6
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_7859.jpg
7
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_8818.jpg
8
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_8230.jpg
9
/content/drive/My Drive/void_detector/data_folders/train_im_s/IMG_8811.jpg
10
/content/drive/My Drive/vo

In [0]:
print(len(os.listdir('/content/drive/My Drive/void_detector/data_folders/test_im_s')))

97


In [0]:
#Parsing the xml file to convert to yolo format data

data_folder =path

frm_fol = ['train_im_bw','val_im_bw','test_im_bw']
des =['bw_train_txt','bw_val_txt','bw_test_txt']
fol = ['bw_train_imgs','bw_val_imgs','bw_test_imgs']

files=['void.xml','no_void.xml']

train_files = os.listdir(os.path.join(data_folder,frm_fol[0]))
val_files = os.listdir(os.path.join(data_folder,frm_fol[1]))
test_files = os.listdir(os.path.join(data_folder,frm_fol[2]))

input_shape=416

for j in range(len(frm_fol)):
	if os.path.exists(data_folder+'/'+des[j])!=True:
		os.mkdir(data_folder+'/'+des[j])
	if os.path.exists(data_folder+'/'+fol[j])!=True:
		print(data_folder+'/'+fol[j])
		os.mkdir(data_folder+'/'+fol[j])

MAPPING = ['empty','non_empty']
for class_no,file in enumerate(files):
	tree = ET.parse(path+'/'+file) 
	root = tree.getroot() 
	for image in root.findall('./images/image'):
		name = image.get('file') 
		name = name.strip().split('/')[-1]
		filename=name.strip().split('.jpg')[0]
		print(filename)
		boxes_ =''
		for box in image.findall('box'):
			left=int(box.get('left'))
			top=int(box.get('top'))
			height=int(box.get('height'))
			width=int(box.get('width'))
			x_min = int(left)
			y_max = int(top)
			x_max = int((left + width))
			y_min = int((top + height))

			if x_min<0:
				x_min = 0
			
			if x_max > input_shape:
				x_max = input_shape
			if y_min < 0:
				y_min=0
			if x_max<x_min:
				temp = x_min
				x_min = x_max
				x_max = temp
			if y_max<y_min:
				temp = y_min
				y_min = y_max
				y_max = temp

			if y_max > input_shape:
				y_max = input_shape
	

			box=name+' '+MAPPING[class_no]+' '+str(x_min)+' '+str(y_min)+' '+str(x_max)+' '+str(y_max)+'\n'
			boxes_+=box
		if name in train_files and boxes_!='':
			with open(os.path.join(data_folder,des[0],filename+'.txt'),'w') as f:
				shutil.copy(os.path.join(data_folder,'train_im_bw',name),os.path.join(data_folder,'bw_train_imgs'))
				f.write(boxes_)
				f.close()

		if name in val_files and boxes_!='':
			with open(os.path.join(data_folder,des[1],filename+'.txt'),'w') as f:
				shutil.copy(os.path.join(data_folder,'val_im_bw',name),os.path.join(data_folder,'bw_val_imgs'))
				f.write(boxes_)
				f.close()
				
		if name in test_files and boxes_!='':
			with open(os.path.join(data_folder,des[2],filename+'.txt'),'w') as f:
				shutil.copy(os.path.join(data_folder,'test_im_bw',name),os.path.join(data_folder,'bw_test_imgs'))
				f.write(boxes_)
				f.close()


/content/drive/My Drive/void_detector/data_folders/bw_train_imgs
/content/drive/My Drive/void_detector/data_folders/bw_val_imgs
/content/drive/My Drive/void_detector/data_folders/bw_test_imgs
1FFB204A-C3CD-4CB2-9ED6-B7964C7E9957
IMG_7405
IMG_7409
IMG_7411
IMG_7412
IMG_7413
IMG_7414
IMG_7415
IMG_7416
IMG_7417
IMG_7418
IMG_7419
IMG_7420
IMG_7421
IMG_7422
IMG_7423
IMG_7424
IMG_7425
IMG_7426
IMG_7427
IMG_7428
IMG_7429
IMG_7430
IMG_7431
IMG_7432
IMG_7433
IMG_7434
IMG_7435
IMG_7436
IMG_7437
IMG_7438
IMG_7439
IMG_7440
IMG_7441
IMG_7442
IMG_7443
IMG_7444
IMG_7445
IMG_7446
IMG_7447
IMG_7448
IMG_7449
IMG_7450
IMG_7451
IMG_7452
IMG_7453
IMG_7454
IMG_7455
IMG_7456
IMG_7457
IMG_7458
IMG_7459
IMG_7460
IMG_7461
IMG_7462
IMG_7463
IMG_7464
IMG_7465
IMG_7466
IMG_7467
IMG_7468
IMG_7469
IMG_7470
IMG_7471
IMG_7472
IMG_7473
IMG_7474
IMG_7475
IMG_7476
IMG_7477
IMG_7478
IMG_7479
IMG_7480
IMG_7481
IMG_7482
IMG_7483
IMG_7484
IMG_7485
IMG_7486
IMG_7487
IMG_7488
IMG_7489
IMG_7490
IMG_7491
IMG_7492
IMG_7493
IMG_74

In [0]:
#Converting the yolo format data to voc format

data_folder = cwd


for name in ['bw_train_','bw_test_','bw_val_']:

    ANNOTATIONS_DIR_PREFIX = os.path.join(data_folder,'data_folders',name+'txt')

    DESTINATION_DIR = os.path.join(data_folder,'data_folders',name+'ann')
    IMAGE_DIR =os.path.join(data_folder,'data_folders',name+'imgs')

    def create_root(file_prefix, width, height):
        root = ET.Element("annotations")
        ET.SubElement(root, "folder").text = "images"
        ET.SubElement(root, "filename").text = "{}.jpg".format(file_prefix)
        ET.SubElement(root, "path").text = IMAGE_DIR+"/{}.jpg".format(file_prefix)
        size = ET.SubElement(root, "size")
        ET.SubElement(size, "width").text = str(width)
        ET.SubElement(size, "height").text = str(height)
        ET.SubElement(size, "depth").text = "3"
        return root


    def create_object_annotation(root, voc_labels):
        for voc_label in voc_labels:
            obj = ET.SubElement(root, "object")
            ET.SubElement(obj, "name").text = voc_label[0]
            ET.SubElement(obj, "pose").text = "Unspecified"
            ET.SubElement(obj, "truncated").text = str(0)
            ET.SubElement(obj, "difficult").text = str(0)
            bbox = ET.SubElement(obj, "bndbox")
            ET.SubElement(bbox, "xmin").text = str(int(voc_label[1]))
            ET.SubElement(bbox, "ymin").text = str(int(voc_label[2]))
            ET.SubElement(bbox, "xmax").text = str(int(voc_label[3]))
            ET.SubElement(bbox, "ymax").text = str(int(voc_label[4]))
        return root

    def create_file(file_prefix, width, height, voc_labels):
        root = create_root(file_prefix, width, height)
        root = create_object_annotation(root, voc_labels)
        tree = ET.ElementTree(root)
        tree.write("{}/{}.xml".format(DESTINATION_DIR, file_prefix))

    def read_file(file_path):
        file_prefix = file_path.split(".txt")[0]
        image_file_name = "{}.jpg".format(file_prefix)
        w, h = 416,416
        with open(ANNOTATIONS_DIR_PREFIX+'/'+file_path, 'r') as file:
            lines = file.readlines()
            voc_labels = []
            for line in lines:
                voc = []
                line = line.strip()
                data = line.split()
                voc.append(data[1])
                voc.append(float(data[2]))
                voc.append(float(data[3]))
                voc.append(float(data[4]))
                voc.append(float(data[5]))
                voc_labels.append(voc)
            create_file(file_prefix, w, h, voc_labels)
        print("Processing complete for file: {}".format(file_path))

    def start():
        if not os.path.exists(DESTINATION_DIR):
            os.makedirs(DESTINATION_DIR)
        for filename in os.listdir(ANNOTATIONS_DIR_PREFIX):
            if filename.endswith('txt'):
                read_file(filename)
            else:
                print("Skipping file: {}".format(filename))

    if __name__ == "__main__":
        start()

Processing complete for file: IMG_7416.txt
Processing complete for file: IMG_7417.txt
Processing complete for file: IMG_7418.txt
Processing complete for file: IMG_7419.txt
Processing complete for file: IMG_7420.txt
Processing complete for file: IMG_7422.txt
Processing complete for file: IMG_7431.txt
Processing complete for file: IMG_7432.txt
Processing complete for file: IMG_7440.txt
Processing complete for file: IMG_7441.txt
Processing complete for file: IMG_7442.txt
Processing complete for file: IMG_7445.txt
Processing complete for file: IMG_7446.txt
Processing complete for file: IMG_7447.txt
Processing complete for file: IMG_7448.txt
Processing complete for file: IMG_7455.txt
Processing complete for file: IMG_7456.txt
Processing complete for file: IMG_7460.txt
Processing complete for file: IMG_7462.txt
Processing complete for file: IMG_7465.txt
Processing complete for file: IMG_7466.txt
Processing complete for file: IMG_7470.txt
Processing complete for file: IMG_7471.txt
Processing 

In [0]:
#Converting the VOC format files to xml files -- to generate tfrecord files

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']

    xml_df = pd.DataFrame(xml_list, columns=column_name)
    names=[]
    for i in xml_df['filename']:
        names.append(i)
    xml_df['filename']=names

    return xml_df

def main():
	for folder in ['bw_train_ann','bw_val_ann','bw_test_ann']:
	    image_path = os.path.join(path, folder)
	    xml_df = xml_to_csv(image_path)
	    xml_df.to_csv(path+'/'+folder+'_labels.csv', index=None)
	    print('Successfully converted xml to csv.')

main()

Successfully converted xml to csv.
Successfully converted xml to csv.
Successfully converted xml to csv.


In [0]:
print(os.path.exists((path+'/'+'bw_train_ann_labels.csv')))
%cd drive/My\ Drive/void_detector
! ls

True
/content/drive/My Drive/void_detector
data_folders  labelmap.pbtxt  model  object_detection  test_results


In [0]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'empty':
        return 1
    if row_label == 'non_empty':
        return 2
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

names=[['train.record','data_folders/bw_train_imgs','data_folders/bw_train_ann_labels.csv'],['val.record','data_folders/bw_val_imgs','data_folders/bw_val_ann_labels.csv']]
print(names[0][0])
def main(_):
  for i in range(2):
    writer = tf.compat.v1.python_io.TFRecordWriter(names[i][0])
    path = os.path.join(names[i][1])
    examples = pd.read_csv(names[i][2])
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(),names[i][0])
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.compat.v1.app.run()


train.record
Successfully created the TFRecords: /content/drive/My Drive/void_detector/train.record
Successfully created the TFRecords: /content/drive/My Drive/void_detector/val.record


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
